### All State Claim Severe
__Peiming (Peter) Huang__


In [1]:
%matplotlib inline
import matplotlib.pyplot as plt

import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error
from sklearn import preprocessing

import xgboost as xgb

def evalerror(preds, dtrain):
    labels = dtrain.get_label()
    return 'mae', mean_absolute_error(np.exp(preds), np.exp(labels))

shift = 200
RANDOM_STATE = 2012

In [2]:
# read in data in both training and test set
train = pd.read_csv("../input/train.csv", nrows=1000)
test  = pd.read_csv("../input/test.csv",  nrows=1000)

train_ids = train['id']
test_ids  = test['id']
y = np.log(train['loss'] + shift)
test['loss'] = np.nan
test_loss = test['loss']

trainX = train.iloc[:, 1:-1]
testX  = test.iloc[:, 1:-1]

trainX = train.drop(['id', 'loss'], 1)
testX  = test.drop(['id', 'loss'], 1)

combined = pd.concat([trainX, testX], axis=0)
print(combined.shape)

combined_cat = combined.select_dtypes(include=['object'])
combined_num = combined.select_dtypes(exclude=['object'])

combined_num['cont1_2_5'] = combined_num['cont1'] + combined_num['cont2'] + combined_num['cont5']
combined_num['cont4_7_10'] = combined_num['cont4'] + combined_num['cont7'] + combined_num['cont10']
combined_num['cont6_8_11'] = combined_num['cont6'] + combined_num['cont8'] + combined_num['cont11']
combined_num['cont9_12'] = combined_num['cont9'] + combined_num['cont12'] 
combined_num['cont3_13_14'] = combined_num['cont3'] + combined_num['cont13'] + combined_num['cont14']
combined_num.drop(['cont1', 'cont2', 'cont3', 'cont4', 'cont5', 'cont6', 'cont7', 'cont8', 'cont9', 
                   'cont10', 'cont11', 'cont12', 'cont13', 'cont14'], 1, inplace=True)

for cat in combined_cat.columns:
    combined_cat[cat], unique = pd.factorize(combined_cat[cat])

joined = pd.concat([combined_cat, combined_num], axis=1)
print("joined shape is", joined.shape)
#print("joined data is", joined.head())

X = joined.iloc[:train_ids.shape[0],:]
#X_test = joined.iloc[train_ids.shape[0]:,:]
#print(X_test.head())
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

# prepare data for XGBoost
dtrain = xgb.DMatrix(X, label=y)
dtest  = xgb.DMatrix(X_test)

dtrain = xgb.DMatrix(X_train, label=y_train)
dtest  = xgb.DMatrix(X_test)

params = { 'min_child_weight': 1,
           'eta': 0.01,
           'colsample_bytree': 0.5,
           'max_depth': 12,
           'subsample': 0.8,
           'alpha': 1,
           'gamma': 1,
           'silent': 1,
           'verbose_eval': True,
           'seed': RANDOM_STATE
         }

print("Before modeling")
clf = xgb.train(params, dtrain, int(2012 / 0.9), feval=evalerror)
prediction = np.log(np.exp(clf.predict(dtest)) - shift)
score = mean_absolute_error(prediction, y_test)
print("The score is ", score)

#print("the comparison is : ", np.column_stack((prediction, y_test)))

submission = pd.DataFrame()
submission['loss'] = prediction
submission['id'] = test_ids
submission.to_csv('peter.csv', index=False)


(2000, 130)


/home/ubuntu/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/home/ubuntu/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/home/ubuntu/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/sta

joined shape is (2000, 121)
Before modeling
The score is  0.427356111437


Here, I try to find out the np.histogram for each categorical values (now numeric) and will draw the graphs (histogram) if it is not binary